<a href="https://colab.research.google.com/github/BYRic-F/Data_practice/blob/main/FB_de_SQL_API_Revision_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

In [1]:
GITHUB_DB_URL = "https://github.com/siara-cc/sakila_sqlite3/raw/master/sakila.db"

In [2]:
import os
import urllib.request
import sqlite3
import pandas as pd

# Nom du fichier local
DB_FILENAME = "sakila.db"

# URL brute corrigée pour télécharger le fichier .db
GITHUB_DB_URL = "https://github.com/siara-cc/sakila_sqlite3/raw/master/sakila.db"

# Si le fichier existe déjà, on le supprime (au cas où il serait corrompu)
if os.path.exists(DB_FILENAME):
    os.remove(DB_FILENAME)

# Téléchargement de la base depuis GitHub
print("Téléchargement de la base SQLite...")
urllib.request.urlretrieve(GITHUB_DB_URL, DB_FILENAME)
print("Téléchargement terminé.")

# Création de la fonction helper pour exécuter des requêtes SQL
def run_query(query):
    conn = sqlite3.connect(DB_FILENAME)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df


Téléchargement de la base SQLite...
Téléchargement terminé.


Schema de sakila :
https://www.jooq.org/img/sakila.png

In [14]:
"""
📌 Question 1 :
Afficher les 10 films les plus longs avec :
- leur titre,
- leur durée,
- leur catégorie,
- un acteur associé à chaque film.

Tables utiles : film, film_category, category, film_actor, actor
"""

query1 = """
SELECT title,
      length,
      c.name as category,
      (FIRST_NAME || ' ' || LAST_NAME) AS actor_name
FROM film f
LEFT JOIN film_category fc ON fc.film_id = f.film_id
LEFT JOIN category c ON c.category_id = fc.category_id
LEFT JOIN film_actor fa ON fa.film_id = f.film_id
LEFT JOIN actor ac ON ac.actor_id = fa.actor_id
GROUP by f.film_id
order by length DESC


LIMIT 10
"""

run_query(query1)


,title,length,category,actor_name
0,CHICAGO NORTH,185,Games,MAE HOFFMAN
1,CONTROL ANTHEM,185,Comedy,BOB FAWCETT
2,DARN FORRESTER,185,Action,BOB FAWCETT
3,GANGS PRIDE,185,Animation,ELVIS MARX
4,HOME PITY,185,Music,KIRSTEN PALTROW
5,MUSCLE BRIGHT,185,Travel,SEAN WILLIAMS
6,POND SEATTLE,185,Animation,DARYL WAHLBERG
7,SOLDIERS EVOLUTION,185,Sci-Fi,JOHNNY LOLLOBRIGIDA
8,SWEET BROTHERHOOD,185,Travel,MILLA KEITEL
9,WORST BANGER,185,Action,PARKER GOLDBERG


In [39]:
"""
📌 Question 2 :
Afficher les 10 clients ayant effectué le plus de locations :
- leur nom complet,
- le nombre total de locations,
- leur classement (ROW_NUMBER),
- le cumul progressif du nombre de locations (fonction SUM OVER).

Tables : rental, customer

HINT : utiliser ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW dans le sum over pour eviter de comptabiliser les total rentals egaux dans la somme cumulée
"""

query2 = """
WITH total_rentals AS (
  SELECT COUNT(RENTAL_id) AS total_rentals,
          customer_id
  FROM rental
  GROUP BY customer_id
  ORDER BY total_rentals DESC
)


SELECT (first_name || ' ' || last_name) as full_name,
        total_rentals,
        ROW_NUMBER() OVER (ORDER BY total_rentals DESC) AS rank,
        SUM(total_rentals) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_rentals
FROM customer c
JOIN total_rentals r ON r.customer_id = c.customer_id
LIMIT 10
"""

run_query(query2)


,full_name,total_rentals,rank,cumulative_rentals
0,ELEANOR HUNT,46,1,46
1,KARL SEAL,45,2,91
2,MARCIA DEAN,42,3,133
3,CLARA SHAW,42,4,175
4,TAMMY SANDERS,41,5,216
5,WESLEY BULL,40,6,256
6,SUE PETERS,40,7,296
7,TIM CARY,39,8,335
8,MARION SNYDER,39,9,374
9,RHONDA KENNEDY,39,10,413


In [81]:
"""
📌 Question 3 :
Afficher l'évolution mensuelle du nombre de films loués pour les 3 catégories les plus populaires.
Pour chaque mois et chaque catégorie :
- le nombre de films loués,
- le nombre du mois précédent,
- la variation (différence).

Tables : rental, inventory, film, film_category, category
Fonction utilisée : LAG (fonction fenêtre)
"""

query3 = """
WITH number_rent AS (
  SELECT count(RENTAL_ID) AS nombre_location,
        strftime('%Y-%m', rental_date) AS rental_month,
        c.name as category
  FROM RENTAL r
  JOIN INVENTORY i ON r.inventory_id = i.inventory_id
  JOIN FILM f ON f.film_id = i.film_id
  JOIN FILM_CATEGORY fc ON fc.film_id = f.film_id
  JOIN CATEGORY c ON c.category_id = fc.category_id
  GROUP BY c.category_id, strftime('%Y-%m', rental_date)
  ORDER BY count(RENTAL_ID) DESC
  ),


most_popular AS (
  SELECT
        c.name as category
  FROM RENTAL r
  JOIN INVENTORY i ON r.inventory_id = i.inventory_id
  JOIN FILM f ON f.film_id = i.film_id
  JOIN FILM_CATEGORY fc ON fc.film_id = f.film_id
  JOIN CATEGORY c ON c.category_id = fc.category_id
  GROUP BY c.category_id
  ORDER BY count(RENTAL_ID) DESC
  LIMIT 3
  )

SELECT rental_month,
       category,
       nombre_location as rental_count,
       LAG(nombre_location) OVER(PARTITION BY category ORDER BY rental_month) AS previous_month,
       nombre_location - (LAG(nombre_location) OVER(PARTITION BY category ORDER BY rental_month)) AS variation

FROM number_rent
WHERE category IN (SELECT category FROM most_popular)

order by category, rental_month

"""

run_query(query3)

,rental_month,category,rental_count,previous_month,variation
0,2005-05,Action,87,NaN,NaN
1,2005-06,Action,160,87.0,73.0
2,2005-07,Action,464,160.0,304.0
3,2005-08,Action,384,464.0,-80.0
4,2006-02,Action,17,384.0,-367.0
5,2005-05,Animation,74,NaN,NaN
6,2005-06,Animation,174,74.0,100.0
7,2005-07,Animation,489,174.0,315.0
8,2005-08,Animation,408,489.0,-81.0
9,2006-02,Animation,21,408.0,-387.0


#API

In [ ]:
"""
Exercice API Nominatim - Géocodage d'adresse

Objectif :
Utiliser l'API Nominatim d'OpenStreetMap pour récupérer les coordonnées GPS (latitude et longitude) d'une adresse donnée.

Consignes :
1. Implémenter une fonction `get_location(address: str) -> dict` qui prend en paramètre une adresse (chaîne de caractères).
2. Cette fonction doit interroger l'API Nominatim via une requête HTTP GET et récupérer la réponse au format JSON.
3. Extraire et retourner un dictionnaire contenant au minimum :
   - 'display_name' : le nom complet normalisé du lieu retourné par l'API,
   - 'lat' : la latitude,
   - 'lon' : la longitude.
4. Gérer le cas où aucune donnée n'est retournée (retourner un dictionnaire vide).
5. Utiliser un header User-Agent approprié pour respecter la politique d'utilisation de l'API.

Exemple d'adresse à tester : "10 Downing Street, London"
"""






In [84]:
import requests
from pprint import pprint  # Import du module pour affichage joli

def get_location(address: str) -> dict:
    """
    Interroge l'API Nominatim pour obtenir les coordonnées GPS d'une adresse.
    Retourne un dictionnaire contenant 'display_name', 'lat' et 'lon'.
    """

    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0',
    'Referer': 'https://www.example.com'}

    link = "https://nominatim.openstreetmap.org/"

    params = {'q' : address,
              'format' : 'json'}

    response = requests.get(link, params= params, headers = headers)

    r = response.json()
    display_name = r[0]['display_name']
    lat = r[0]['lat']
    lon = r[0]['lon']

    result = {'display_name' : display_name,
              'lat' : lat,
              'lon' : lon}

    return result


# Exemple d'utilisation
adresse = "10 Downing Street, London"
resultat = get_location(adresse)

print("Résultat formaté :")
pprint(resultat)


Résultat formaté :
{'display_name': '10 Downing Street, 10, Downing Street, Westminster, Covent '
                 'Garden, City of Westminster, Greater London, England, SW1A '
                 '2AA, United Kingdom',
 'lat': '51.5034878',
 'lon': '-0.1276965'}
